# The aim of this notebook is to build a classifier for the REDHAT predicting business value competition. We will 1) Do some data clean-up with Pandas 2) Use the tf.feature_columns api to apply transformations to the data, and 3) Use Keras to train a Feed-forward Neural Network.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
people = pd.read_csv('/kaggle/input/predicting-red-hat-business-value/people.csv.zip')
activites_train = pd.read_csv('/kaggle/input/predicting-red-hat-business-value/act_train.csv.zip')
activites_test = pd.read_csv('/kaggle/input/predicting-red-hat-business-value/act_test.csv.zip')


## The people and activities table can be linked via shared column people_id. Merge the people set with the train and test activities sets to create the full train and test set

In [ ]:
def merge_sets(people, activities):
    return activities.join(people.set_index('people_id'), on='people_id', lsuffix='_activity', rsuffix='_person')

train = merge_sets(people, activites_train)
test = merge_sets(people, activites_test)

train.shape, test.shape

In [ ]:
submission_ids = test.activity_id

## After some data exploration it is noticable that some categorical columns of the activities table only refer to specific types of activities. Instances that represent these types of activities have values for those columns. Instances of a different activity type do not have a value for these columns. Therefore, many activity instances have missing values. We will simply fill these with the value 'missing'.

In [ ]:
train.isnull().sum()

In [ ]:
def preprocess(dataset):
    dataset = dataset.drop(['date_person', 'date_activity', 'people_id', 'activity_id'], axis=1) # 
    dataset = dataset.fillna('missing')
    return dataset

train = preprocess(train)
test = preprocess(test)

# Let us look at how many categories the categorical columns have. We can see that some columns such as group_1 or even char_4_person have many categories. One-hot-encoding these categories would result in too many columns. Therefore, we will use embeddings on columns with more than 12 categories. Let us first extract the column names we will use for one-hot-encoding, embedding, and numerical input.

In [ ]:
train.dtypes # Quick look at the different column types before checking categories

In [ ]:
categorical_data = train.select_dtypes(include=['object'])
categorical_data.nunique()

In [ ]:
column_names = categorical_data.columns
print("CATEGORICAL COLUMNS : \n" + str(column_names))

embed_column_indicators = categorical_data.nunique() > 12
onehot_column_indicators = categorical_data.nunique() <= 12

EMBED_COLUMN_NAMES = column_names[embed_column_indicators]
ONEHOT_COLUMN_NAMES = column_names[onehot_column_indicators]
NUM_COLUMN_NAMES = train.select_dtypes(exclude=['object']).columns

print("\nEMBED COLUMNS : \n" + str(EMBED_COLUMN_NAMES))
print("\nONEHOT COLUMNS : \n" + str(ONEHOT_COLUMN_NAMES))
print("\nNUMERICAL COLUMNS : \n" + str(NUM_COLUMN_NAMES))

Hardcode the column names to keep them robust from potential future data changes

In [ ]:
EMBED_COLUMN_NAMES = ['char_1_activity', 'char_2_activity', 'char_8_activity',
       'char_9_activity', 'char_10_activity', 'group_1', 'char_3_person',
       'char_4_person', 'char_7_person']
ONEHOT_COLUMN_NAMES = ['activity_category', 'char_3_activity', 'char_4_activity',
       'char_5_activity', 'char_6_activity', 'char_7_activity',
       'char_1_person', 'char_2_person', 'char_5_person', 'char_6_person',
       'char_8_person', 'char_9_person']
NUM_COLUMN_NAMES = ['char_10_person', 'char_11', 'char_12', 'char_13', 'char_14',
       'char_15', 'char_16', 'char_17', 'char_18', 'char_19', 'char_20',
       'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26',
       'char_27', 'char_28', 'char_29', 'char_30', 'char_31', 'char_32',
       'char_33', 'char_34', 'char_35', 'char_36', 'char_37', 'char_38']

## Take a quick look at the maximum values of the numerical columns, so that we know to normalize them later on.

In [ ]:
np.amax(train[NUM_COLUMN_NAMES], axis=0)

Since all numerical columns apart from char_38 and the outcome are booleans, let's have a quick look at their distribution, in case we notice anything special.

In [ ]:
train[NUM_COLUMN_NAMES].astype('float32').hist(figsize=(32,32))

# Finally, lets create the bridge of our input pipeline that will turn columns of our Pandas DataFrame into input of our upcoming Neural Network. For this we will use the tf.feature_columns API, that allows us to specify which transformations to perform on each column before feeding the columns to a Neural Network.

# Most importantly, this API allows for more compatability. The code we write with it will allow us to use our Pandas DataFrame set as an input for training, but pretty much the same code will allow us to also use other data sources such as cross-platform TFRecord files at test time. It simply expects a tf.data.Dataset object.

Before anything however, quickly create a validation set.

In [ ]:
trainset, valset = train_test_split(train, test_size=0.2)
testset = test
print(len(trainset), 'train examples')
print(len(valset), 'validation examples')
print(len(testset), 'test examples')
print(type(trainset),type(valset),type(testset))

In [ ]:
def dataframe_to_dataset(dataframe, shuffle=False, batch_size=32, train=False, buffer_size=50000):
    dataframe = dataframe.copy()
    if train:
        labels = dataframe.pop('outcome')
    else:
        labels = tf.ones(shape=dataframe.shape[0]) * -1 # Will be ignored at test time
        
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        buffer_size = buffer_size if buffer_size < dataframe.shape[0] else dataframe.shape[0]
        dataset = dataset.shuffle(buffer_size, seed=42)
        
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
train_dataset = dataframe_to_dataset(trainset, train=True, shuffle=True, batch_size=4096)
val_dataset = dataframe_to_dataset(valset, train=True, batch_size=4096)
test_dataset = dataframe_to_dataset(testset, batch_size=4096)

We will create a definition of what to do with each column.

In [ ]:
feature_columns = []

for column_name in NUM_COLUMN_NAMES:
    feature_columns.append(tf.feature_column.numeric_column(key=column_name, dtype=tf.float32, default_value=-1))
    
for column_name in ONEHOT_COLUMN_NAMES:
    vocabulary = train[column_name].unique()
    categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
                          key=column_name, vocabulary_list=vocabulary, default_value=-1)
    
    onehot_column = tf.feature_column.indicator_column(categorical_column)
    feature_columns.append(onehot_column)
    
for column_name in EMBED_COLUMN_NAMES:
    vocabulary = train[column_name].unique()
    categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
                          key=column_name, vocabulary_list=vocabulary, default_value=-1)
    
    embedding_dimensions = 16 if len(vocabulary) < 500 else 256
    embed_column = tf.feature_column.embedding_column(categorical_column, dimension=embedding_dimensions)
    feature_columns.append(embed_column)

## The feature_columns list now contains all the specification that our keras model needs to take a tf.data.Dataset and properly process each column before feeding it into the model. We could use this same specification to parse TFRecord files that are structured the same as our Pandas Dataframe.

In [ ]:
print(feature_columns[0],'\n')
print(feature_columns[35],'\n')
print(feature_columns[48],'\n')

# All we need to do is use the following layer as the first layer of our keras model.

In [ ]:
processing_layer = keras.layers.DenseFeatures(feature_columns)

# Finally, lets build and train our model

In [ ]:
model = keras.models.Sequential()
model.add(processing_layer)
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(16, kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.01)))

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Nadam(0.01),
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_plateau = keras.callbacks.ReduceLROnPlateau(patience=3)

In [ ]:
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[early_stopping, reduce_lr_plateau], steps_per_epoch=20, validation_steps=20)

# Some extra code for submitting

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.epoch, history.history["lr"], "bo-")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate", color='b')
plt.tick_params('y', colors='b')
plt.gca().set_xlim(0, 10 - 1)
plt.grid(True)

ax2 = plt.gca().twinx()
ax2.plot(history.epoch, history.history["val_loss"], "r^-")
ax2.set_ylabel('Validation Loss', color='r')
ax2.tick_params('y', colors='r')

plt.title("Reduce LR on Plateau", fontsize=14)
plt.show()

In [ ]:
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0.7, 1)
plt.show()

In [ ]:
val_preds = model.predict(val_dataset, verbose=True)

In [ ]:
from sklearn.metrics import roc_auc_score
y_true = list(val_dataset.map(lambda x,y: y).unbatch().as_numpy_iterator())
roc_auc_score(y_true, val_preds)

In [ ]:
test_preds = model.predict(test_dataset, verbose=True)

In [ ]:
np.mean(test_preds), test_preds[0], test_preds

In [ ]:
ids = np.array(submission_ids).reshape((-1,)) # make 1-dimensional
preds = test_preds.reshape((-1,)) # make 1-dimensional
print(ids.shape, preds.shape)

my_submission = pd.DataFrame({'activity_id': ids, 'outcome': preds})

my_submission.to_csv('REDHAT_submission_2.csv', index=False)